  # Introdução

Este notebook foi feito por Gabriel Matz visando o estudo dirigido para a prova de certificação oficial para o Tensorflow. A descrição da prova está no seguinte link:
https://www.tensorflow.org/certificate?hl=pt-br.

Utilizei como base os seguintes notebooks(além de sites variados e stack overflow para pequenas consultas):

1- https://github.com/mrdbourke/tensorflow-deep-learning

2-https://github.com/williamcwi/DeepLearning.AI-TensorFlow-Developer-Professional-Certificate

O conteúdo deste terceiro notebook engloba a terceira parte de exigências da prova do Tensorflow, que estão descritas abaixo:

"(3) Classificação de Imagens
Você precisa compreender como construir modelos de reconhecimento de imagem e detecção de objetos com redes neurais profundas e redes neurais convolucionais usando o TensorFlow 2.x. Você precisa saber como:

❏ Definir redes neurais convolucionais com camadas Conv2D e camadas de pooling.

❏ Construir e treinar modelos para processar conjuntos de dados de imagem do mundo real.

❏ Entender como usar convoluções para aprimorar sua rede neural.

❏ Utilizar imagens do mundo real em diferentes formas e tamanhos.

❏ Aplicar aumentação de imagem para evitar overfitting.

❏ Utilizar o ImageDataGenerator.

❏ Compreender como o ImageDataGenerator rotula imagens com base na estrutura do diretório."

# Desenvolvimento

**❏ Definir redes neurais convolucionais com camadas Conv2D e camadas de pooling.**




**❏ Entender como usar convoluções para aprimorar sua rede neural.**

Fontes que mais gostei para entender o conteúdo:



1.   https://en.wikipedia.org/wiki/Convolutional_neural_network
2.   https://poloclub.github.io/cnn-explainer/ - Meu favorito. Possui ferramentas de vizualização

A tabela abaixo do Daniel Bourke apresenta as noções básicas:

| **Hyperparameter/Layer type** | **What does it do?** | **Typical values** |
| ----- | ----- | ----- |
| Input image(s) | Target images you'd like to discover patterns in| Whatever you can take a photo (or video) of |
| Input layer | Takes in target images and preprocesses them for further layers | `input_shape = [batch_size, image_height, image_width, color_channels]` |
| Convolution layer | Extracts/learns the most important features from target images | Multiple, can create with [`tf.keras.layers.ConvXD`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) (X can be multiple values) |
| Hidden activation | Adds non-linearity to learned features (non-straight lines) | Usually ReLU ([`tf.keras.activations.relu`](https://www.tensorflow.org/api_docs/python/tf/keras/activations/relu)) |
| Pooling layer | Reduces the dimensionality of learned image features | Average ([`tf.keras.layers.AvgPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)) or Max ([`tf.keras.layers.MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)) |
| Fully connected layer | Further refines learned features from convolution layers | [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) |
| Output layer | Takes learned features and outputs them in shape of target labels | `output_shape = [number_of_classes]` (e.g. 3 for pizza, steak or sushi)|
| Output activation | Adds non-linearities to output layer | [`tf.keras.activations.sigmoid`](https://www.tensorflow.org/api_docs/python/tf/keras/activations/sigmoid) (binary classification) or [`tf.keras.activations.softmax`](https://www.tensorflow.org/api_docs/python/tf/keras/activations/softmax) |




**❏ Construir e treinar modelos para processar conjuntos de dados de imagem do mundo real.**

Resolverei nesse notebook o problema do Food101(classificação múltipla).

In [ ]:
!nvidia-smi -L # checa se está usando gpu. Se estiver no collab, confira o ambiente de execução

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-9d9bdfc6-536f-6f8b-f61e-56d9b4a99def)


In [ ]:
import tensorflow as tf

print(tf.__version__)

2.15.0


https://www.kaggle.com/datasets/dansbecker/food-101

In [ ]:
import os
input_directory = '/content'
print(os.listdir(input_directory))
if "food-101" in os.listdir():
    print("Dataset já foi baixado")
else:
    print("Downloading ...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting..")
    !tar xzvf food-101.tar.gz > /dev/null 2>&1
    print("Extraction done!")

['.config', 'sample_data']
--2024-03-29 23:51:39--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2024-03-29 23:51:39--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  19.3MB/s    in 4m 30s  

2024-03-29 23:56:09 (17.7 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]

Dataset downloaded!
Extracting..
Extraction done!


In [ ]:
# Lista os arquivos e diretórios
for dirpath, dirnames, filenames in os.walk("food-101"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 2 images in 'food-101'.
There are 101 directories and 0 images in 'food-101/images'.
There are 0 directories and 1000 images in 'food-101/images/ceviche'.
There are 0 directories and 1000 images in 'food-101/images/foie_gras'.
There are 0 directories and 1000 images in 'food-101/images/prime_rib'.
There are 0 directories and 1000 images in 'food-101/images/bread_pudding'.
There are 0 directories and 1000 images in 'food-101/images/cup_cakes'.
There are 0 directories and 1000 images in 'food-101/images/tacos'.
There are 0 directories and 1000 images in 'food-101/images/beet_salad'.
There are 0 directories and 1000 images in 'food-101/images/pulled_pork_sandwich'.
There are 0 directories and 1000 images in 'food-101/images/mussels'.
There are 0 directories and 1000 images in 'food-101/images/peking_duck'.
There are 0 directories and 1000 images in 'food-101/images/oysters'.
There are 0 directories and 1000 images in 'food-101/images/apple_pie'.
There are 0 dir

Pré-processamento


1.   Separar em teste e treino
2.   Usar o ImageDataGenerator para ajustar em tensores

Opcional: tf.keras.utils.image_dataset_from_directory. Segundo o próprio Tensorflow ImageDataGenerator será deprecado.(feito nos extras)



In [ ]:
import pathlib
data_dir = pathlib.Path('/content/food-101/images/')
category_files = {}
for category_dir in data_dir.iterdir():
    if category_dir.is_dir():
        category_name = category_dir.name
        category_files[category_name] = list(category_dir.glob('*'))



**❏ Utilizar imagens do mundo real em diferentes formas e tamanhos.**



O conjunto de dados tem forma diferente. Você precisa padronizar.

In [ ]:
# Hiperparâmetros
batch_size = 32
img_height = 224
img_width = 224



**❏ Utilizar o ImageDataGenerator.**




**❏ Compreender como o ImageDataGenerator rotula imagens com base na estrutura do diretório.**

Você pode uar o ImageDataGenerator a partir dos diretórios

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                shear_range=0.2,
                                                                zoom_range=0.2, # métodos de aumento de dados
                                                                horizontal_flip=True,
                                                                validation_split=0.2)
train = train_datagen.flow_from_directory(
    '/content/food-101/images',
    target_size=(img_height, img_width),
    shuffle=True, # IMPORTANTE
    batch_size=32,
    class_mode='sparse',  # Se quiser one hot é 'categorical'
    subset='training'
)

val = train_datagen.flow_from_directory(
    '/content/food-101/images',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


Agora que temos nossas entradas corretamente formatadas, montamos a rede neural.

In [ ]:
num_classes=101

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(), # passando pra 1 dimensão
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.sparse_categorical_crossentropy,
  metrics=['accuracy'])

model.fit(
  train,
  epochs=10,
  validation_data= val
)

Epoch 1/10
 202/2525 [=>............................] - ETA: 13:48 - loss: 4.6276 - accuracy: 0.0122

KeyboardInterrupt: 


**❏ Aplicar aumentação de imagem para evitar overfitting.**

O aumento de dados pode ser feito tanto por camadas de aumento como(no extra coloquei um exemplo):

https://www.tensorflow.org/tutorials/images/data_augmentation?hl=pt-br

Ou usando o próprio ImageDataGenerator, como feito acima com o shear_range, zoom_range e flip.

# Sem usar o ImageDataGenerator

**Usando o tf.keras.utils.image_dataset_from_directory**

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=1,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True)

Found 101000 files belonging to 101 classes.
Using 80800 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=1,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True)

Found 101000 files belonging to 101 classes.
Using 20200 files for validation.


Tuning e prefetching para melhorar o desempenho

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE) #
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
val_ds, train_ds

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

In [ ]:
num_classes=101

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.RandomFlip("horizontal_and_vertical"), # o aumento de dados é feito com camadas
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2525/2525 [==============================] - 45s 17ms/step - loss: 4.4033 - accuracy: 0.0390 - val_loss: 4.0978 - val_accuracy: 0.0782
Epoch 2/10
2525/2525 [==============================] - 34s 14ms/step - loss: 3.9632 - accuracy: 0.1014 - val_loss: 3.8162 - val_accuracy: 0.1248
Epoch 3/10
2525/2525 [==============================] - 34s 14ms/step - loss: 3.7213 - accuracy: 0.1415 - val_loss: 3.6749 - val_accuracy: 0.1512
Epoch 4/10
2525/2525 [==============================] - 35s 14ms/step - loss: 3.5690 - accuracy: 0.1655 - val_loss: 3.5726 - val_accuracy: 0.1736
Epoch 5/10
2525/2525 [==============================] - 35s 14ms/step - loss: 3.4738 - accuracy: 0.1842 - val_loss: 3.5026 - val_accuracy: 0.1816
Epoch 6/10
2525/2525 [==============================] - 35s 14ms/step - loss: 3.3976 - accuracy: 0.1963 - val_loss: 3.4437 - val_accuracy: 0.1946
Epoch 7/10
2522/2525 [============================>.] - ETA: 0s - loss: 3.3389 - accuracy: 0.2052

Perceba como é muito mais rápido